In [1]:
import gym
from tqdm.notebook import tqdm
COLAB = False
if not COLAB:
    import os
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.layers import *
from tensorflow.keras import Input
from utils import ReplayBuffer, OrnsteinUhlenbeckActionNoise
path_base = "models/"
RESUME = False

In [28]:
import gym
from gym import spaces
import numpy as np
import pandas as pd
import json
import datetime as dt

MAX_Money = 100000
class StockEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    
    def __init__(self,dfs, train, number=1, **kwargs):
        super(StockEnv,self).__init__()
        self.train = train
        self.MAX_shares = 2147483647
        self.Min_Brokerage = 30
        self.count = number
        self.Brokerage_rate = 0.001
        
        if "balance" in kwargs.keys():
            Max_Money = kwargs["balance"]
        if "Max_Shares" in kwargs.keys():
            self.MAX_shares = kwargs["Shares"]
        if "Broke_limit" in kwargs.keys():
            self.Min_Brokerage = kwargs["Broke_limit"]
        if "Broke_rate" in kwargs.keys():
            self.Brokerage_rate = kwargs["Broke_rate"]
        
        self.dfs = dfs
        self.action_space = spaces.Box(low = np.array([-1]), high = np.array([1]), dtype = np.float16)
        lower = [0]*number
        higher = [1]*number
        self.observation_space = spaces.Box(low=np.array(lower),high=np.array(higher),dtype=np.float32)
    
    def _get_prices(self):
#         print ("Day {0}".format(self.df.loc[self.current_step,"Date"]))
#         print ("low: {0} high: {1}".format(self.df.loc[self.current_step,"Open"],self.df.loc[self.current_step,"Close"]))
        return np.array([np.random.uniform(df.loc[self.current_step,"Open"], df.loc[self.current_step,"Close"]) for df in self.dfs])
    
    def _observe(self, prices):
        frame = prices
        frame = frame / self.highest_price
        info = {
            'balance' : self.balance,
            'highest_price': self.highest_price,
            'current_price': self.current_prices,
            #'time': self.df.loc[self.current_step,'time_stamp'],
            'shares_held': self.shares_held,
            'max_worth': self.max_net_worth,
            'broke_limit': self.Min_Brokerage,
            'broke_rate': self.Brokerage_rate
        }
        
        return frame, info
        
    def reset(self,balance = MAX_Money,**kwargs):
        if "balance" in kwargs.keys():
            Max_Money = kwargs["balance"]
        if "Max_Shares" in kwargs.keys():
            self.MAX_shares = kwargs["Shares"]
        if "Broke_limit" in kwargs.keys():
            self.Min_Brokerage = kwargs["Broke_limit"]
        if "Broke_rate" in kwargs.keys():
            self.Brokerage_rate = kwargs["Broke_rate"]
        
        if self.train:
            self.current_step = np.random.randint(0,len(self.dfs[0].loc[:,'Open'].values)-1)
        else:
            self.current_step = 0
       
        self.balance = balance
        self.shares_held = np.array([0]*self.count)
        self.current_prices = self._get_prices() 
        self.net_worth = self.balance + sum(self.shares_held*self.current_prices)
        self.initial_worth = self.net_worth
        self.max_net_worth = self.net_worth
        self.highest_price = np.max(self.current_prices)
        frame,_ =  self._observe(self.current_prices)
        return frame
    
    def _broke(self,amount):
        return max(amount * self.Brokerage_rate,self.Min_Brokerage)
    
    def _take_action(self, action_vector):
        self.current_prices = self._get_prices()
        self.highest_price = max(self.highest_price,np.max(self.current_prices))
        action_vector  = action_vector*self.MAX_shares
        for i in range(self.count):
            if action_vector[i] < 0:
                # sell
                action_vector[i] = -1*action_vector[i]
                if action_vector[i] > self.shares_held[i]:
                    action_vector[i] = self.shares_held[i]
                amount_gained = action_vector[i]*self.current_prices[i]
                broke = self._broke(amount_gained)
                amount_gained -= broke
                if self.balance + amount_gained < 0:
                    a1 = np.floor(self.balance/((self.Brokerage_rate-1)*self.current_prices[i]))
                    action = np.floor(-(self.balance-self.Min_Brokerage)/self.current_prices[i])
                    if self._broke(a1*self.current_prices[i]) == a1*self.current_prices[i]*self.Brokerage_rate:
                        action_vector[i] = max(a1,action_vector[i])
                    action_vector[i] = max(action_vector[i],0)
                    amount_gained = action_vector[i]*self.current_prices[i]
                    amount_gained -= self._broke(amount_gained)
                self.balance +=amount_gained
                self.shares_held[i] = self.shares_held[i]-action_vector[i]
            elif action_vector[i]>0:
                #buy
                amount_required = self.current_prices[i]*action_vector[i] + self._broke(self.current_prices[i]*action_vector[i])
                if amount_required > self.balance:
                    a1 = np.floor(self.balance/((self.Brokerage_rate+1)*self.current_prices[i]))
                    action_vector[i] = np.floor((self.balance-self.Min_Brokerage)/self.current_prices[i])
                    if self._broke(a1*self.current_prices[i]) == a1*self.current_prices[i]*self.Brokerage_rate:
                        action_vector[i] = max(a1,action_vector[i])
                    action_vector[i] = max(action_vector[i],0)
                    amount_required = action_vector[i]*self.current_prices[i]
                    amount_required -= self._broke(amount_required)
                self.balance -= amount_required
                self.shares_held[i] += action_vector[i]
        reward = self.balance + sum(self.shares_held* self.current_prices) - self.net_worth
        self.net_worth = self.balance + sum(self.shares_held* self.current_prices)
        if self.net_worth > self.max_net_worth:
            self.max_net_worth = self.net_worth
        return reward, self.current_prices
            
    def step(self, action):
        reward, prices = self._take_action(action)
        self.current_step+=1
        if self.current_step > len(self.dfs[0].loc[:,'Open'].values)-1:
            self.current_step = 0
        
        done = self.net_worth<=0
        obs, info = self._observe(prices)
        
        return obs, reward, done, info
    
    def render(self, mode='human', close = False):
        profit = self.net_worth - self.initial_worth
        print(f'Step: {self.current_step}')
        print(f'Net Worth:{self.net_worth}')
        print(f'Profit: {profit}')


def create_stock_env(locations, train=True):
    dfs = [pd.read_csv(location) for location in locations]
    for df in dfs:
        (df.sort_values("Date"))
    return StockEnv(dfs, train, len(locations)), dfs[0].shape[0]



In [29]:
class Actor:
    def __init__(self, params):
        self.output_range = params["output_range"]
        self.hidden_layers = params["actor_hidden_layers"]
        self.state_dimensions = params["state_dimensions"]
        self.action_dimensions = params["action_dimensions"]
        self.actor = self.model()
        
    def model(self):
        inputs = Input(shape=(1, self.state_dimensions))
        x = Lambda(lambda x: x)(inputs)
        for layer in self.hidden_layers:
            x = Dense(layer, activation='relu')(x)
        x = Dense(self.action_dimensions, activation='tanh')(x)
        x = Lambda(lambda x: x*self.output_range)(x)
        model = tf.keras.Model(inputs = inputs, outputs = x)
        return model
    
    def get_action(self, state):
        state_tensor = tf.Variable(shape = state.shape, initial_value = state)
        return (self.actor(state_tensor)).numpy()

    def save_weights(self):
        self.actor.save_weights(path_base + "actor.h5")
               
    def load_weights(self):
        self.actor.load_weights(path_base + "actor.h5")
        
    
class Critic:
    def __init__(self, params):
        self.hidden_layers = params["critic_hidden_layers"]
        self.state_dimensions = params["state_dimensions"]
        self.action_dimensions = params["action_dimensions"]
        self.optimizer = params["critic_optimizer"]
        self.critic_online = self.model()
        self.critic_target = self.model()


    def model(self):
        input_a = Input(shape = (1, self.state_dimensions))
        input_b = Input(shape = (1, self.action_dimensions))
        x = concatenate([input_a, input_b], axis=-1)
        for layer in self.hidden_layers:
            x = Dense(layer, activation='relu')(x)
        x = Dense(1, activation='linear')(x)
        model = tf.keras.Model(inputs=[input_a, input_b], outputs = x)
        model.compile(loss='mse', optimizer=self.optimizer)
        return model
    
    def save_weights(self):
        self.critic_online.save_weights(path_base + "critic_online.h5")
        self.critic_target.save_weights(path_base +  "critic_target.h5")
               
    def load_weights(self):
        self.critic_online.load_weights(path_base + "critic_online.h5")
        self.critic_target.load_weights(path_base + "critic_target.h5")

    def get_qvalues(self, state_array, action_array, online=True):
        state_tensor = tf.Variable(shape = state_array.shape, initial_value = state_array)
        action_tensor = tf.Variable(shape = action_array.shape, initial_value = action_array)
        return (self.critic_online([state_tensor, action_tensor]).numpy() if online else self.critic_target([state_tensor, action_tensor]).numpy())
    
    def call(self, state_tensor, action_tensor, online = True):
        return (self.critic_online([state_tensor, action_tensor]) if online else self.critic_target([state_tensor, action_tensor]))
    def merge_networks(self, tau):
        self.critic_target.set_weights(tau*np.array(self.critic_online.get_weights())
                                                                    + (1-tau)*np.array(self.critic_target.get_weights()))
        

In [38]:
class Agent:
    def __init__(self, params, test=False):
        self.test = test
        self.actor = Actor(params)
        self.critic = Critic(params)
        self.buffer = ReplayBuffer(params["buffer_size"])
        self.state_dimensions = params["state_dimensions"]
        self.action_dimensions = params["action_dimensions"]
        self.discount = params["discount"]
        self.action_range = params["output_range"]
        self.save_frequency = params["save_frequency"]
        self.batch_size = params["batch_size"]
        self.optimizer = params["actor_optimizer"]
        self.tau = params["tau"]
        self.step = 0
        self.noise_func =  OrnsteinUhlenbeckActionNoise(mu=np.zeros(params["action_dimensions"]))
        if RESUME:
            self.load_networks()
        
    def agent_start(self, observation):
        observation = np.reshape(observation, (1, self.state_dimensions))
        act = np.squeeze(self.actor.get_action(observation))
        if not self.test:
            for i in range(act.shape[0]):
                act[i] = np.squeeze(self.clip_action((act[i] + self.noise_func())[0]))
        else:
            for i in range(act.shape[0]):
                act[i] = np.squeeze(self.clip_action(act[i]))
        self.prev_state = observation
        self.prev_action = act
        return act

    def clip_action(self, action):
        if abs(action) > self.action_range:
            action *= abs(self.action_range)/abs(action)
        
        return action

    def agent_step(self, reward, observation):
        observation = np.reshape(observation, (1, self.state_dimensions))
        if not self.test:
            relay = (self.prev_state, self.prev_action, reward, observation)
            self.buffer.add(relay)
        self.prev_state = observation
        act = np.squeeze(self.actor.get_action(observation))
        if not self.test:
            for i in range(act.shape[0]):
                act[i] = np.squeeze(self.clip_action((act[i] + self.noise_func())[0]))
            self.train(self.batch_size)
        else:
            for i in range(act.shape[0]):
                act[i] = np.squeeze(self.clip_action(act[i]))
        
        self.prev_action = act
        return self.prev_action 
    
    def save_networks(self):
        self.actor.save_weights()
        self.critic.save_weights()

    def load_networks(self):
        self.actor.load_weights()
        self.critic.load_weights()


    def train(self, sample_size):
        self.step += 1
        batch, batch_size = self.buffer.sample(sample_size)

        state_array = np.array([ element[3] for element in batch])
        action_array = self.actor.get_action(state_array)
        prev_state_array = np.array([ element[0] for element in batch])
        prev_action_array = np.array([ [[element[2]]] for element in batch])
        output = self.critic.get_qvalues(state_array, action_array, False)
        output = np.array([element[2] + self.discount*out[0] for element, out in zip(batch, output)])
        self.critic.critic_online.fit([state_array, action_array], output, verbose=0)

        prev_state_tensor = tf.Variable(shape = prev_state_array.shape, initial_value = prev_state_array)
        prev_action_tensor = tf.Variable(shape = prev_action_array.shape, initial_value = prev_action_array)

        with tf.GradientTape(persistent=True) as g:
            g.watch(prev_action_tensor) 
            g.watch(prev_state_tensor)
            value = self.critic.call(prev_state_tensor, prev_action_tensor)
            action = self.actor.actor(prev_state_tensor)
            
        gradient = -tf.squeeze(g.gradient(value, prev_action_tensor))
        gradient = tf.cast(gradient, tf.float32)
        gradient_actor = g.gradient(action, self.actor.actor.trainable_weights, gradient)
        gradient_actor = list(map(lambda x: tf.math.divide(x, batch_size), gradient_actor))
        self.optimizer.apply_gradients(zip(gradient_actor, self.actor.actor.trainable_weights))
        self.critic.merge_networks(self.tau)

        if self.step%self.save_frequency == 0:
            self.save_networks()



In [39]:
AGENT_PARAMS = {
	"output_range": 1,
	"actor_hidden_layers": [60, 16],
	"critic_hidden_layers": [60, 16],
	"state_dimensions": 29,
	"action_dimensions": 29,
	"critic_optimizer": tf.keras.optimizers.Adam(learning_rate = 0.001),
	"actor_optimizer": tf.keras.optimizers.Adam(learning_rate = 0.0001),
	"batch_size": 64,
	"buffer_size":1000000,
	"discount": 0.99,
	"tau": 0.001,
	"save_frequency": 100
}

In [40]:
import os
files = ["train/" + f for f in os.listdir("train/")]

In [41]:
env, stamps = create_stock_env(files)

In [42]:
y = env.reset()

In [43]:
y

array([0.35321339, 0.39572086, 0.21128619, 0.50573689, 0.24027794,
       0.48543738, 0.54282275, 0.469029  , 0.95749591, 0.31165276,
       0.75146567, 0.4360287 , 0.21634115, 0.47254771, 0.15718709,
       0.66458452, 0.54935278, 0.18102918, 0.64808163, 0.27973536,
       0.30922683, 0.46164503, 0.52661648, 1.        , 0.13033785,
       0.32989886, 0.48528385, 0.45077625, 0.25508728])

In [44]:
env.step(y)

(array([0.35410773, 0.39586117, 0.2113022 , 0.50352776, 0.24048512,
        0.48505872, 0.54411025, 0.46943028, 0.95431472, 0.31196466,
        0.75236174, 0.43602572, 0.21635295, 0.47195392, 0.15709856,
        0.66363665, 0.54937729, 0.18132001, 0.64715962, 0.27972499,
        0.30994882, 0.46097375, 0.52658894, 0.99975684, 0.13037384,
        0.32993829, 0.48543199, 0.45078721, 0.25463368]),
 939.9289812043135,
 False,
 {'balance': 93.070719564354,
  'highest_price': 192.8908944820442,
  'current_price': array([ 68.30415667,  76.35801585,  40.75827094,  97.12592065,
          46.38738898,  93.56340966, 104.95391245,  90.54882714,
         184.07862064,  60.17514175, 145.12372903,  84.10539197,
          41.73251465,  91.0356132 ,  30.30288241, 128.00946733,
         105.96987675,  34.97497988, 124.83119744,  53.95640372,
          59.78630418,  88.91763907, 101.57421258, 192.84399167,
          25.1479264 ,  63.64209218,  93.6354102 ,  86.9527473 ,
          49.11651816]),
  'shares

In [45]:
agent = Agent(AGENT_PARAMS)
ITERATIONS = 20000
pbar = tqdm(desc="Iteration: ", total=ITERATIONS)
action = agent.agent_start(env.reset())
observation, reward, done, info = env.step(action)
profit = np.zeros(ITERATIONS)
prev_profit = 0
for _ in range(ITERATIONS):
    action = agent.agent_step(reward, observation)
    observation, reward, done, info = env.step(action)
    profit[i] += reward + prev_profit
    prev_profit += reward
    pbar.update(1)

InvalidArgumentError: Matrix size-incompatible: In[0]: [1,30], In[1]: [58,60] [Op:MatMul]